In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima_model import ARMA
from sklearn.model_selection import train_test_split
from statsmodels.formula.api import ols
from sklearn import linear_model

df=pd.read_excel("Complete-dataset-FINAL.xlsx")

### Note
Dit model is een alternatief voor de lineare regressie model. De algemene bunsiness & data understanding zijn te vinden in de Jupyter notebook van de lineare regressie. In deze notebook zal er gefocust worden op het voorspellingsmodel (ARIMA model).

# Business understanding
We gaan het ARIMA model gebruiken. Er wordt als eerste gekeken naar hoe stationair de data is. Dit zal betekenen dat een tijdsreeks niet afhankelijk is van tijd. En dan zullen de parameters voor het ARIMA model worden bepaald. ARIMA(p=?, d=?,q=?)

### ARIMA model
Het doel van een ARIMA model is het nabootsen van een tijdsserie. Dit wordt gedaan door de variaties in de data te modelleren door middel van de volgende drie opties: 

- AR (auto-regressief), voorgaande waardes worden gebruikt om nieuwe waardes te voorspellen.
- I (integrated), niet de originele serie maar een gedifferentieerde tijdsserie wordt gebruikt. Dit om de tijdsserie stationair te maken. 
- MA (moving average), voorafgaande fouten worden gebruikt om nieuwe fouten voorspellen. Dit heeft een smoothing effect, een bewegend gemiddelde. 

Door deze effecten te mengen kun je de meeste tijdsseries nabootsen. In de volgende sectie gaan we de effecten van de AR en MA termen op een tijdsserie bekijken. 



# Data preparation

In [ ]:
df=df.dropna()
df.columns = df.columns.str.replace('Total Error', 'Total')
df.columns

In [ ]:
df.Year.astype('int32')
df['Year'] =pd.to_datetime(df.Year, format='%Y')
df.info()

In [ ]:
df = df.set_index('Year')
df.head()

In [ ]:
# predict only top 10 manufacturers
list_top10 = df['Manufacturer'].value_counts()[:10].index.tolist()
top10_manufacturers = df.loc[df['Manufacturer'].isin(list_top10)]
top10_manufacturers['Manufacturer'].value_counts()
top10_manufacturers.info()

In [ ]:
manufacturers = top10_manufacturers.groupby("Manufacturer")
manufacturers.size().nlargest(20)

In [ ]:
top10_manufacturers = top10_manufacturers.dropna()
top10_manufacturers=top10_manufacturers[['Manufacturer', 'Total']]
top10_manufacturers.info()

In [ ]:
for name, data in manufacturers:
    print(name)
    plt.plot(data.index, data['Total'])
    plt.show()

# Note
Hier is te zien dat er niet een duidelijke lijn wordt weergegeven van door de jaren heen. Dit is een indicatie dat het ARIMA model mogelijk niet geschikt zal zijn.

# Modeling : parameters ARIMA

In [ ]:
# top10_manufacturers= top10_manufacturers.dropna()
# manufacturers = top10_manufacturers.groupby("Manufacturer")

In [ ]:
# Check for stationarity of the time-series data
# We will look for p-value. In case, p-value is less than 0.05, the time series data can said to have stationarity.
from statsmodels.tsa.stattools import adfuller
List_meetmethoden = []


#AIC input is to compute the optimal number iteratively.
for name, data in manufacturers: 
    data= data.dropna()
    print(name) 
    df_stationarityTest = adfuller(data['Total'], autolag='AIC')    
    
    data['Total'].plot(figsize=(16,10))
    plt.show() 
    print("\n")
    print(f'ADF Statistic: {df_stationarityTest[0]}')
    print(f'n_lags: {df_stationarityTest[1]}')
    print(f'p-value: {df_stationarityTest[1]}')
    for key, value in df_stationarityTest[4].items():
        print('Critial Values:')
        print(f'   {key}, {value}')    

    
    if df_stationarityTest[1] > 0.05:
        List_meetmethoden.append(name)
    print("\n")

print("Meetmethoden met een p-waarden van boven de 0.05:")
List_meetmethoden

### Analyse van p-values
Een p-value van 0.05 of er onder ligt, betekent dat de data stationair is. De resultaten laten zien dat bij sommige de meetmethoden de data niet stationair is. Dit betekent dat bij deze meetmethoden nullhypothese (voorspelt geen effect of relatie) mogelijk van toepassing is.
Om deze meetmethoden stationair te maken, wordt het gedifferentieerd. 

#### Differentiatie van meetmethoden met p-value > 0.05

In [ ]:
#Één keer diffrentiëren
for name, data in manufacturers:
    if name in List_meetmethoden:
        print("\n" + name) 
        df_stationarityTest = adfuller(data['Total'].diff().dropna(), autolag='AIC')   
        data['Total'].diff().plot(figsize=(16,10))
        plt.show() 
        
        print("")
        print(f'ADF Statistic: {df_stationarityTest[0]}')
        print(f'n_lags: {df_stationarityTest[1]}')
        print(f'p-value: {df_stationarityTest[1]}')
        for key, value in df_stationarityTest[4].items():
            print('Critial Values:')
            print(f'   {key}, {value}')    

        

In [ ]:
#2x diff = I (ARIMA)
for name, data in manufacturers:
    if name in List_meetmethoden:
        print("\n")
        print(name) 
        # data['Total'].diff().diff().plot(figsize=(16,10))
        # plt.show()
        data_diff = data['Total'].diff().diff().dropna()
        
        df_stationarityTest = adfuller(data_diff, autolag='AIC')    

        print(f'ADF Statistic: {df_stationarityTest[0]}')
        print(f'n_lags: {df_stationarityTest[1]}')
        print(f'p-value: {df_stationarityTest[1]}')
        for key, value in df_stationarityTest[4].items():
            print('Critial Values:')
            print(f'   {key}, {value}')    



# Conclusie p-waarden
Hier is te zien dat bij 2 keer differentiëren dat de p-waarden juist groter worden. Voor het ARIMA model wordt de kleinste p-waarde gebruikt. Daarom nemen we 0 als parameter bij alle meetmethoden. Dus ARIMA(p=?,d=0,q=?)

# Data preparation ACF & PACF

### ACF plot

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
for name, data in manufacturers: 
    acf =plot_acf(data['Total'], title=name, alpha=.05)
    

### Analyse van ACF plot
In dit autocorrelatieplot liggen bijna alle waardes niet binnen het lichtblauwe onzekerheidsgebied (deze variantie in autocorrelatie kan mogelijk worden verklaard door ruis). Dit betekent dat we juist wel MA-parameter moeten in te stellen (datapunten buiten het onzekerheidgebied).
Per methode is er een andere MA waarden. 

In [ ]:
Lijst_MA = [13,2,11,13,26,5,15,8,7,2]

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf

for name, data in manufacturers: 
    pacf = plot_pacf(data['Total'], title=name)

### Analyse van pacf plot
De partiële autocorrelatie geeft weer hoe sterk het verband is tussen de waarde van een lag en de waarde van voorgaande lags. Als er tussen de lags nog significante correlatie bestaat is dit een aanwijzing dat er auto-correlatie optreed en dat het instellen van de AR parameter een goed idee is.

Bij alle grafieken is te zien dat de eerste lag het meest significant is. Daarom nemen we p met de waarde 1. Dus ARIMA(p=1,d=0,q=lijst_MA)

# ARIMA modeling

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import datetime

## ARIMA summary output

In [ ]:
counter= 0

for name, data in manufacturers:
    print(name)
    data.index = pd.DatetimeIndex(data.index).to_period('M')

    model = sm.tsa.arima.ARIMA(data['Total'].diff().dropna(), order=(0,2,Lijst_MA[counter]))
    model_fit = model.fit()
    print(model_fit.summary())
    counter = counter +1
    
   
#P>|z| significant? onder 0.01

## ARIMA prediction visualisation

In [ ]:
counter= 0
from sklearn.model_selection import train_test_split

import datetime
for name, data in manufacturers:
    tempdf = data[['Total']].dropna()
    X = tempdf.iloc[:, :1].values
    Y = tempdf.index.values
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=0)

    model = sm.tsa.arima.ARIMA(X_train, order=(0,2,Lijst_MA[counter]))
    model_fit = model.fit()
    counter = counter +1 #next in list

    y_pred = model_fit.predict(disp=0, exog=None, dynamic=False)
    length_predicted_values = len(y_pred)


    preddf = pd.DataFrame(y_pred, columns=['Total'])
    preddf['Year'] = pd.to_datetime('2022-01-01')
    preddf = preddf.set_index('Year')
    
    
    # Add tempdf and preddf together and reset index
    tempdf = pd.concat([tempdf, preddf], axis=0)

    tempdf.reset_index(inplace=True)
    
    #find predicted value in concatenated dataframe
    start_index_predValues = max(tempdf.index)-length_predicted_values    
    real_values_df = tempdf.loc[:start_index_predValues]
   
    #sort by year and reset index so all years line up
    real_values_df = real_values_df.sort_values(by="Year")
    real_values_df = real_values_df.reset_index()
    
    l = sns.lineplot(x=real_values_df.index, y='Total', data=real_values_df, hue=real_values_df['Year'])
    l.set_title('Total error prediction: ' + name )
    
    #show predicted values
    predict_values_df = tempdf.loc[start_index_predValues:]
    sns.lineplot(x=predict_values_df.index, y='Total', data=predict_values_df, color='orange')

    sns.set(rc = {'figure.figsize':(25,8)})
    plt.show()
    